RNN for sections classifier using only layout for three classes

In [ ]:
import tensorflow as tf
from tensorflow.contrib import keras
from tensorflow.contrib.keras import models
from tensorflow.contrib.keras import datasets
from tensorflow.contrib.keras import layers
from tensorflow.contrib.keras import preprocessing
from tensorflow.contrib.keras import backend as K
from tensorflow.contrib.keras import callbacks
from tensorflow.contrib.keras import utils
from datetime import datetime

from collections import defaultdict
import pandas as pd
import numpy as np
import os
import json
import pickle
import sys
import random
import glob
from sklearn import metrics,cross_validation
import csv
from collections import Counter
import warnings
import string
import re
import nltk
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

learn = tf.contrib.learn
csv.field_size_limit(sys.maxsize)
tf.logging.set_verbosity(tf.logging.WARN)

In [ ]:
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.80
#config.gpu_options.allow_growth=True
config.gpu_options.visible_device_list="0"
session = tf.Session(config=config)
keras.backend.set_session(session)
K.gpu_setup = ["gpu0"]

In [ ]:
MAX_DOCUMENT_LENGTH=100
maxlen_word=20
num_classes=3
embedding_dims=100
filters=250
kernel_size = 3
hidden_dims = 100
batch_size=128

In [ ]:
model_dir="rnn_only_layout_sections_classifier_3_classes/"

if not os.path.exists(model_dir):
    os.makedirs(model_dir)

model_dir_three ="../section_header_cnn_keras/cnn_only_text_sections_classifiers_3_classes/"


sys.stdout = open(model_dir+'report_sections_classifier_three_classes_cnn_only_layout_'+ 
                  str(datetime.now()).replace(" ","-").replace(":","-")+'.txt', 'w')

n_files=60
n_epoch=100
print("Number of files: ")
print(str(n_files))
print("\nNumber of epoch: ")
print(str(n_epoch))
print("\n")


In [ ]:
# load data for three classes
char_processor = pickle.load(open(model_dir_three+"char_processor",
                                 'rb'))
x_train_char = pickle.load(open(model_dir_three+"x_train_char",
                                 'rb'))
x_test_char = pickle.load(open(model_dir_three+"x_test_char",
                                 'rb'))

x_train_layout = pickle.load(open(model_dir_three+"x_train_layout",
                                 'rb'))
x_test_layout = pickle.load(open(model_dir_three+"x_test_layout",
                                 'rb'))

y_train = pickle.load(open(model_dir_three+"y_train",
                                 'rb'))
y_test = pickle.load(open(model_dir_three+"y_test",
                                 'rb'))


In [ ]:
print('Build model for layout...')
max_layout=x_train_layout.shape[1]

input_layout =  layers.Input(shape=(max_layout,), name = 'input_layout')

embedding_layout = layers.Embedding(output_dim=embedding_dims, input_dim=max_layout, 
                                     input_length=max_layout)(input_layout)

gru_layout=layers.GRU(128,activation='tanh', recurrent_activation='hard_sigmoid', use_bias=True)(embedding_layout)

dense_hidden_layout = layers.Dense(hidden_dims)(gru_layout)

dropout_layout= layers.Dropout(0.2)(dense_hidden_layout)
activation_layout = layers.Activation("tanh")(dropout_layout)



In [ ]:
main_output = layers.Dense(num_classes, activation='softmax', name='main_output')(activation_layout)
model = models.Model(inputs=[input_layout], outputs=[main_output])


In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print model.summary()

In [ ]:
tb_callback = callbacks.TensorBoard(log_dir=model_dir, histogram_freq=50, write_graph=True,
                          embeddings_freq=50,embeddings_layer_names=None)

In [ ]:
filepath=model_dir+"weights-improvement-{epoch:02d}-{val_acc:.9f}.hdf5"
checkpoint = callbacks.ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
history = callbacks.History()

In [ ]:
model.fit(x_train_layout, y_train,
          batch_size=batch_size,shuffle=True,
          epochs=n_epoch,verbose=2,
          validation_data=(x_test_layout, y_test),callbacks=[checkpoint,history])


In [ ]:
model.save(model_dir+"final_model")

In [ ]:
score = np.asarray(model.predict(x_test_layout)).argmax(1)
predict = np.round(np.asarray(model.predict(x_test_layout))).argmax(1)
targ = y_test.argmax(1)


In [ ]:
print(metrics.classification_report(targ, predict))
print metrics.confusion_matrix(targ, predict)


In [ ]:
print "Done"
